In [ ]:
import json

import numpy as np
from scipy.spatial import KDTree
from scipy.optimize import linear_sum_assignment

In [ ]:
#np.random.seed(42)

def random_spots_in_radius(n_spots, n_dim, radius):
    """
    get random relative (integer) coordinates within radius
    """
    
    if np.isscalar(radius):
        radius = np.array([radius] * n_dim)
    else:
        radius = np.array(radius)
    
    res_spots = []
    while len(res_spots) < n_spots:
        # uniformly distributed on hypersquare
        candidate = [np.random.randint(-radius[i],radius[i]+1) for i in range(len(radius))]
        # reject spots not in hypersphere
        if (np.sum(np.array(candidate)**2 / radius**2) <= 1):
            res_spots.append(candidate)

    return np.array(res_spots)

radius = 100
n_dim = 3
n_spots = 10
spots = random_spots_in_radius(n_spots, n_dim, radius)

def get_descriptor(spot, spots, neighbors_for_descriptor = 3):
    kd = KDTree(spots)
    ds, idxs = kd.query(spot, k=neighbors_for_descriptor+1)
    d_vecs = spot - spots[idxs[1:]]
    desc = d_vecs.flatten()
    return desc

def match_kd(descs_a, descs_b):
    kd = KDTree(descs_a)
    ds, idxes = kd.query(descs_b)
    return idxes

def match_la(descs_a, descs_b):
    
    descs_a = np.array(descs_a)
    descs_b = np.array(descs_b)
        
    n_spots_max = np.max([descs_a.shape[0], descs_b.shape[0]])
    n_spots_a = descs_a.shape[0]
    n_spots_b = descs_b.shape[0]
    
    # pad for empty assignment
    if (n_spots_b > n_spots_a):
        descs_a = np.concatenate((descs_a, descs_b[descs_a.shape[0]:]))
    if (n_spots_b < n_spots_a):
        descs_b = np.concatenate((descs_b, descs_a[descs_b.shape[0]:]))
    
    print (descs_a)
    print (descs_b)
    a = np.tile(descs_a, [n_spots_max, 1] )
    b = np.repeat(descs_b, n_spots_max, 0)
    ds = np.sqrt(np.sum((a-b)**2, axis=1)).reshape((n_spots_max, n_spots_max))

    _, idxes_la = linear_sum_assignment(ds)
    idxes_la[idxes_la>=n_spots_a] = -1
    return idxes_la[:n_spots_b]

descs_gt = [get_descriptor(s, spots) for s in spots[:int(n_spots//1.2)]]
perm = np.arange(n_spots)
np.random.shuffle(perm)
spots_t = spots[perm]

#spots_t = np.concatenate(( spots_t, spots[n_spots//2:]))

spots_t += np.array([210] * n_dim)
#print(spots_t)
descs_shuff = [get_descriptor(s, spots_t) for s in spots_t]

idxes = match_kd(descs_gt, descs_shuff)
idxes_la = match_la(descs_gt, descs_shuff)

print(perm)
#print(idxes)
print(idxes_la)

(np.sum(idxes_la[idxes_la!=-1]!=perm), np.sum(perm!=idxes))

In [ ]:
descs_a = np.array([[1,1],[2,2],[22,22], [3,3]])
descs_b = np.array([[12,12], [1,1],[2,2], [3,3], [12,12]])


a_pad = np.concatenate((descs_a[:descs_b.shape[0]], descs_b[descs_a.shape[0]:]))
a = np.tile(a_pad,[5,1]) - np.repeat(descs_b,5,0)

np.sum(a**2, axis=1).reshape((5,5))
#descs_b[descs_a.shape[0]:]
#descs_a

#match_la( descs_a, descs_b, )
#a_pad